In [1]:
# Utility
import re, sys, copy, arrow, random
from helpers import *

In [2]:
def scratch():
    re_cid = re.compile("^(\d+)(cm|in)$")
    print(re_cid.findall("123in"))
        
scratch()

[('123', 'in')]


In [3]:
@timeit
def day5():
    raw = contents(2020, 5, 1)
    
    @timeit
    def part1():
        pass
    part1()
    
    @timeit
    def part2():
        pass
    part2()
day5()

'part1'  0.00 ms
'part2'  0.00 ms
'day5'  0.60 ms


In [4]:
@timeit
def day4():
    raw = contents(2020, 4, 1)
    
    re_fields = re.compile("(byr|iyr|eyr|hgt|hcl|ecl|pid|cid):(\S+)")
    re_cid = re.compile("cid:\S+")
    
    rules = {
        "byr": { 
            "reg": re.compile("^\d{4}$"), 
            "valid": lambda v: 1920 <= int(v) <= 2002 
        },
        "iyr": { 
            "reg": re.compile("^\d{4}$"), 
            "valid": lambda v: 2010 <= int(v) <= 2020 
        },
        "eyr": { 
            "reg": re.compile("^\d{4}$"), 
            "valid": lambda v: 2020 <= int(v) <= 2030 
        },
        "hgt": { 
            "reg": re.compile("^(\d+)(cm|in)$"), 
            "valid": lambda r: (59 <= int(r[0]) <= 76) if (r[1] == "in") else (150 <= int(r[0]) <= 193) 
        },
        "hcl": { 
            "reg": re.compile("^#[a-f0-9]{6}$"),     
            "valid": always(True)
        },
        "ecl": { 
            "reg": re.compile("^(?:amb|blu|brn|gry|grn|hzl|oth)$"), 
            "valid": always(True)
        },
        "pid": { 
            "reg": re.compile("^\d{9}$"), 
            "valid": always(True)
        },
    }
    
    def get_full(lines):
        hold = []
        for line in lines:
            if line == '':
                yield " ".join(hold)
                hold = []
                continue
            hold.append(line)
        yield " ".join(hold)
    
    def validate(lines, secondary=None):        
        for passport in get_full(lines):
            entries = re_fields.findall(passport)
            count = len(entries)
            has_cid = re_cid.search(passport) is not None
            
            if (count == 8) or (count == 7 and has_cid is False):
                yield 1 if (secondary is None or secondary(entries) is True) else 0
            else:
                yield 0
                
    @timeit
    def part1():
        print(sum(p for p in validate(raw)))
    part1()
    
    def secondary(entries):
        for code, data in entries:
            if code in rules:
                rule = rules[code]
                inp = rule["reg"].findall(data)
                if len(inp) == 0 or rule["valid"](inp[0]) is False:
                    return False
        return True
    
    @timeit
    def part2():
        print(sum(p for p in validate(raw, secondary)))
    part2()
day4()

245
'part1'  1.42 ms
133
'part2'  2.73 ms
'day4'  5.46 ms


In [5]:
@timeit
def day3():
    from helpers.grids.rc import data_to_grid, chess_move
    
    grid, mc, mr = data_to_grid(contents(2020, 3, 1))
    
    def traverse(movement):
        current = origin
        trees = 0
        mod = mc + 1
        
        while True:
            r, c = add(current, movement)
            if r > mr:
                return trees
            current = (r, c % mod)
            trees += 1 if grid[current] == "#" else 0
            
    mul = lambda lst: reduce(lambda x, y: x * y, lst)
            
    @timeit
    def part1():
        print(traverse(chess_move(rights=3, downs=1)))
    part1()
    
    @timeit
    def part2():
        moves = [
            chess_move(rights=1, downs=1),
            chess_move(rights=3, downs=1),
            chess_move(rights=5, downs=1),
            chess_move(rights=7, downs=1),
            chess_move(rights=1, downs=2),
        ]
        
        trees = [traverse(m) for m in moves]
        print(mul(trees))
    part2()
        
day3()

299
'part1'  0.46 ms
3621285278
'part2'  1.74 ms
'day3'  7.66 ms


In [6]:
@timeit
def day2():
    re_split = re.compile("[\d\w]+")
    
    def process(lines):
        for rule in lines:
            n1, n2, l, p = re_split.findall(rule)
            yield int(n1), int(n2), l, p
    
    raw = contents(2020, 2, 1)
    lines = list(process(raw))
    
    @timeit
    def part1():
        valid = 0
        for low, high, letter, pw in lines:
            valid += 1 if low <= pw.count(letter) <= high else 0
        print(valid)
    part1()
    
    @timeit
    def part2():
        valid = 0
        for p1, p2, letter, pw in lines:
            l1, l2 = pw[p1-1], pw[p2-1]
            valid += 1 if (l1 == letter or l2 == letter) and (l1 != l2) else 0
        print(valid)
    part2()
        
day2()

445
'part1'  0.64 ms
491
'part2'  0.41 ms
'day2'  5.69 ms


In [7]:
@timeit
def day1():
    data = map_ints(contents(2020, 1, 1))
    first = second = False
    for a in data:
        for b in data:
            if first is False and a+b == 2020:
                first = True
                print(a, b, a*b)
                
            for c in data:
                if second is False and a+b+c == 2020:
                    second = True
                    print(a, b, c, a*b*c)
                if first and second:
                    break
day1()

736 169 1115 138688160
1768 252 445536
'day1'  90.18 ms
